In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import random

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import statistics
from torch.utils.data import TensorDataset,DataLoader

In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 15.8 MB/s 
     |████████████████████████████████| 287 kB 60.6 MB/s 


In [ ]:
import contractions

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install fasttext
import fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 5.1 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3164515 sha256=a39a43720d4c25b56242007371d7bb655547816887dc709d854fd81ae37ed483
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import string
string.punctuation
import re
import fasttext.util

In [ ]:
df_neg = pd.read_csv(r"/content/drive/MyDrive/DAN/neg.txt",sep='\t',header=None)
df_pos = pd.read_csv(r"/content/drive/MyDrive/DAN/pos.txt",sep='\t',header=None)
df_neg.columns=['review']
df_pos.columns=['review']
lst_0 = [0] * len(df_neg)
lst_1=[1]*len(df_pos)
df_neg['sentiment'] = lst_0
df_pos['sentiment']=lst_1
df=pd.concat([df_neg,df_pos], ignore_index=True)

SyntaxError: ignored

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
class cleaning:
  def __init__(self, df):
    self.df = df
  def remove_punctuation(self,text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
  def remove_html(self,data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data
  def remove_url(self,data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data
  def remove_stopwords(self,text):
    
    filtered_sentence = remove_stopwords(text)
    return filtered_sentence
        

  def clean_data(self):
    self.df['review']= self.df['review'].apply(lambda x:self.remove_punctuation(x))
    self.df['review']=self.df['review'].apply(lambda z: self.remove_html(z))
    self.df['review']=self.df['review'].apply(lambda z: self.remove_url(z))
    self.df['review']=self.df['review'].apply(lambda z: contractions.fix(z) )
    # self.df['review']= df['review'].apply(lambda x:self.remove_stopwords(x))
   
    self.df['review']= self.df['review'].apply(lambda x: x.lower())
    self.df['review'] = self.df.apply(lambda row: nltk.word_tokenize(row['review']), axis=1)
    # self.df['review']= df['review'].apply(lambda x:self.remove_stopwords(x))
    return self.df

  
  

In [ ]:
obj=cleaning(df)
df=obj.clean_data()



In [ ]:
fasttext.util.download_model('en', if_exists='ignore')
ft = fasttext.load_model('cc.en.300.bin')


In [ ]:
embeddings=[]
for i in range(len(df['review'])):
  
  list=[]
  for j in range(len(df['review'][i])):
    word=df['review'][i][j]
   
    vector=ft.get_word_vector(word)
    list.append(vector)
  embeddings.append(np.mean(list,axis=0))
  



In [ ]:
embeddings_df=pd.DataFrame(embeddings)
embeddings_df = embeddings_df.join(df['sentiment'])


In [ ]:
train, val = train_test_split(embeddings_df, test_size=0.2)


In [ ]:
train_x= train.iloc[:,:300].to_numpy()
train_y=train.iloc[:,-1].to_numpy()
val_x= val.iloc[:,:300].to_numpy()
val_y=val.iloc[:,-1].to_numpy()

In [ ]:
train_x =torch.from_numpy(train_x)
train_y =torch.from_numpy(train_y)
val_x =torch.from_numpy(val_x)
val_y =torch.from_numpy(val_y)


In [ ]:
train_dataset=TensorDataset(train_x,train_y)

In [ ]:
val_dataset=TensorDataset(val_x,val_y)

In [ ]:
input_size = 300
hidden_size = [128, 64]
num_classes = 2
num_epochs = 100
batch_size = 100
learning_rate = 0.01 

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=True)

In [ ]:
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                            batch_size=batch_size, 
                                            shuffle=True)

In [ ]:
class NeuralNet(nn.Module):
     def __init__(self, input_size, hidden_size, num_classes):
         super(NeuralNet, self).__init__()
         self.input_size = input_size
         self.l1 = nn.Linear(input_size, hidden_size[0]) 
         self.relu1 = nn.Tanh()
         self.l2 = nn.Linear(hidden_size[0], hidden_size[1])
         self.relu2=nn.Tanh()
         self.l3 = nn.Linear(hidden_size[1],num_classes)
         self.softmax=nn.Softmax(dim=1)
     def forward(self, x):
         out = self.l1(x)
         out = self.relu1(out)
         out = self.l2(out)
         out = self.relu2(out)
         out = self.l3(out)
         out = self.softmax(out)
       
         return out
        


In [ ]:
model = NeuralNet(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
     train_loss=0
     val_loss=0
     acc=0
     for i, (review, sentiment) in enumerate(train_loader):
        outputs = model(review)
        loss = criterion(outputs, sentiment)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss=train_loss+loss.item()
        # print (f'Epoch [{epoch+1}/{num_epochs}], Step[{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

     for batch in val_loader:
          x, y = batch
          outputs = model(x)
          loss = criterion(outputs, sentiment)
          # optimizer.zero_grad()
          val_loss=val_loss+loss.item()
          out = torch.argmax(outputs, dim=1)
         
          acc =acc+ torch.sum(out == y)

     
     print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {val_loss / len(val_loader)}')
     print("accuracy on validation dataset: ",(acc/len(val_dataset)))